In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
import time

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
tf.config.experimental.list_physical_devices(device_type = None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]

In [4]:
f16 = pd.read_csv('data/f16_l3.csv')

In [5]:
X = f16['Force']
y = f16['Acceleration3']

In [6]:
#X = MinMaxScaler().fit_transform(X.values.reshape(-1,1))

In [7]:
#Declare the number of model iterations we will run 
test_iterations = 250

#Declare the size of the initial train test split
test_size = 0.20

#Declare the number of hidden layers to add
layers = 2

#Declare the density of the hidden layers
density = 64

#Declare the type of activation for the hidden layers
activation = tf.keras.layers.LeakyReLU(alpha = 0.03)

#Assign the loss function the model will use to train
loss = 'mean_squared_error'

#Assign the optimizer for the compilitation
optimizer = 'adam'

#Declare the patience count used for early stopping
patience = 25

#Declare the test size used for early stopping train test split
es_test_size = 0.25

#Declare the batch size for use in the model
batch_size = 64

#Declare the maximum number of epochs for our model
epochs = 250

In [8]:
X_train, X_true, y_train, y_true = train_test_split(
    X, y, random_state = 8669, test_size = test_size)

In [9]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:1'):
    model = Sequential()

    #Creates a for loop that will add the number of layers based on the variable declared in the previous cell    
    for i in range(layers):
        #Adds a model layer with density and activation based on the variables declared in the previous cell
        model.add(Dense(density, activation = activation))
    
    model.add(Dense(1, activation='linear'))
    
    #model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

In [10]:
es = EarlyStopping(monitor = 'loss', patience = patience, restore_best_weights = True)
mc = ModelCheckpoint(filepath = 'test_model.h5', monitor = 'loss', save_best_only=True)
X_es_train, X_es_true, y_es_train, y_es_true = train_test_split(X_train, y_train, test_size = es_test_size, random_state = 8669)

In [11]:
#Creates a dataframe by which we will eventually put in our list created above
model_record = pd.DataFrame(columns = ['model_num', 'loss_type', 'time', 'epochs', 'r2', 'mae', 'mse', 'rmse', 'max_error'])

#Creates a dataframe by which our model's predicted values and true values will be stored
predict_record = pd.DataFrame(y_true).reset_index(drop = True)

#Creates a numpy array by which the for loop will use to count model runs and is then used to name df columns
model_counter = np.array([0])

In [12]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:1'):
    
#Performs model training repeatedly based on the variable declared previously    
    for i in range(test_iterations):
        model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

        #Creates an empty list for storing model metrics and other information 
        record_list = list() 
        #Starts a counter that adds 1 everytime a fitting is performed
        model_counter = model_counter + 1
        #Starts a timer to end after the round of fitting is complete
        start_time = datetime.now()
        
        #Fits our model/batch_size and epochs are declared previously
        #model.fit(x = X_train, y = y_train.values, 
        #          batch_size = batch_size, epochs = epochs)
        
        history = model.fit(x = X_es_train, y = y_es_train.values, 
                  validation_data = (X_es_true, y_es_true),
                  batch_size = batch_size, epochs = epochs, 
                  callbacks = [es, mc])
        
        #Saves our model predictions
        y_pred = model.predict(X_true)
        
        #Saves our model run #, time to run, and model metrics to our temporary list
        record_list.extend([len(model_record)+1, #A model iteration count
                            loss, #Records the loss metric used to fit the model
                            format(datetime.now() - start_time), #Calculates the time it took to complete the model
                            len(history.history['loss']), #The epoch number of our model checkpoint 
                            r2_score(y_true, y_pred), #R2
                            mean_absolute_error(y_true, y_pred), #MAE
                            mean_squared_error(y_true, y_pred), #MSE
                            np.sqrt(mean_squared_error(y_true, y_pred)), #RMSE
                            max_error(y_true, y_pred) #MaxError
                           ])        
        
        #Adds the temporary list of model metrics (etc) to the end of a dataframe
        model_record.loc[len(model_record)] = record_list
        
        #Converts our predictions to a dataframe so it will play nice
        y_pred_df = pd.DataFrame(y_pred)
        
        #Adds predictions as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, y_pred_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'm' + str(model_counter[0])})
        
        #Calculates the residual values for each prediction and stores it as a dataframe 
        residuals_df = pd.DataFrame(abs(predict_record.iloc[:,len(predict_record.columns)-1] - predict_record.iloc[:,0]))
        
        #Adds y residuals as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, residuals_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'res' + str(model_counter[0])})

Epoch 1/250
1006/1006 [==============================] - 4s 4ms/step - loss: 0.2456 - mse: 0.2456 - val_loss: 0.2418 - val_mse: 0.2418
Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2399 - mse: 0.2399 - val_loss: 0.2723 - val_mse: 0.2723
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2373 - mse: 0.2373 - val_loss: 0.2393 - val_mse: 0.2393
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2346 - mse: 0.2346 - val_loss: 0.2541 - val_mse: 0.2541
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2339 - mse: 0.2339 - val_loss: 0.2390 - val_mse: 0.2390
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2317 - mse: 0.2317 - val_loss: 0.2435 - val_mse: 0.2435
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2292 - mse: 0.2292 - val_loss: 0.2364 - val_mse: 0.2364
Epoch 8/250
1006/1006 [==============================] 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 62/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2099 - mse: 0.2099 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 63/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2192 - val_mse: 0.2192
Epoch 64/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2093 - mse: 0.2093 - val_loss: 0.2183 - val_mse: 0.2183
Epoch 65/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 66/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2094 - mse: 0.2094 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 67/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2096 - mse: 0.2096 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 68/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2186 - val_mse: 0.2186
Epoch 122/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2180 - val_mse: 0.2180
Epoch 123/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2087 - mse: 0.2087 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 124/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 125/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 126/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2201 - val_mse: 0.2201
Epoch 127/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 128/250
1006/1006 [==============================

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2180 - val_mse: 0.2180
Epoch 19/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2208 - val_mse: 0.2208
Epoch 20/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 21/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 22/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 23/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2189 - val_mse: 0.2189
Epoch 24/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 25/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2180 - val_mse: 0.2180
Epoch 79/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 80/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2194 - val_mse: 0.2194
Epoch 81/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2199 - val_mse: 0.2199
Epoch 82/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2216 - val_mse: 0.2216
Epoch 83/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 84/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 85/250
1006/1006 [==============================] - 3s 

...

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 24/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 25/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 26/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 27/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 28/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 29/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 30/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 18/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 19/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 20/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 21/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 22/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 23/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 24/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2175 - val_mse: 0.2175
Epoch 38/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 39/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 40/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 41/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 42/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2183 - val_mse: 0.2183
Epoch 43/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 44/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 49/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 50/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 51/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 52/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 53/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 54/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 55/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 47/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 48/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 49/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 50/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 51/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 52/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 53/250
1006/1006 [==============================] - 3s 

In [13]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:08:28.749627,163,0.193524,0.227772,0.215562,0.464287,5.221067
1,2,mean_squared_error,0:06:00.526209,115,0.191869,0.229129,0.216004,0.464763,5.225255
2,3,mean_squared_error,0:02:08.908339,41,0.180627,0.234919,0.219009,0.467984,5.210839
3,4,mean_squared_error,0:01:43.736878,34,0.192083,0.228802,0.215947,0.464701,5.228512
4,5,mean_squared_error,0:04:38.374819,92,0.190100,0.228490,0.216477,0.465271,5.228271
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:01:21.764186,29,0.191598,0.229714,0.216077,0.464841,5.215133
246,247,mean_squared_error,0:01:53.418644,40,0.191198,0.229151,0.216184,0.464956,5.230335
247,248,mean_squared_error,0:02:17.624583,49,0.191406,0.228599,0.216128,0.464896,5.205252
248,249,mean_squared_error,0:02:53.244353,62,0.190895,0.230101,0.216265,0.465043,5.227360


In [14]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.102292,0.212598,-0.105198,0.209692,-0.098584,0.216306,-0.109584,0.205306,-0.108231,...,-0.094493,0.220397,-0.109584,0.205306,-0.084682,0.230208,-0.106797,0.208093,-0.099642,0.215248
1,-0.267150,-0.294532,0.027382,-0.324543,0.057393,-0.413557,0.146407,-0.335163,0.068013,-0.272466,...,-0.257924,0.009226,-0.266622,0.000528,-0.277524,0.010374,-0.356965,0.089815,-0.298852,0.031702
2,0.257710,0.322510,0.064800,0.256768,0.000942,0.229179,0.028531,0.317112,0.059402,0.271670,...,0.328483,0.070773,0.282201,0.024491,0.277131,0.019421,0.281943,0.024233,0.259567,0.001857
3,-0.193330,-0.097400,0.095930,-0.101043,0.092287,-0.089837,0.103493,-0.104651,0.088679,-0.104192,...,-0.090732,0.102598,-0.105898,0.087432,-0.080874,0.112456,-0.102980,0.090350,-0.095924,0.097406
4,-0.409560,-0.131309,0.278251,-0.132399,0.277161,-0.128265,0.281295,-0.140663,0.268897,-0.132565,...,-0.113730,0.295830,-0.127115,0.282445,-0.108316,0.301244,-0.132336,0.277224,-0.122928,0.286632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.085033,0.063796,-0.083031,0.061794,-0.065891,0.044654,-0.094501,0.073264,-0.084985,...,-0.074353,0.053116,-0.086325,0.065088,-0.061591,0.040354,-0.084991,0.063754,-0.074971,0.053734
21447,-0.264320,0.100697,0.365017,0.078130,0.342450,0.081205,0.345525,0.095084,0.359404,0.081306,...,0.088925,0.353245,0.067965,0.332285,0.107754,0.372074,0.082551,0.346871,0.092144,0.356464
21448,0.060178,-0.103086,0.163264,-0.106952,0.167130,-0.103130,0.163308,-0.113062,0.173240,-0.108888,...,-0.095098,0.155276,-0.110177,0.170355,-0.085295,0.145473,-0.107426,0.167604,-0.100240,0.160418
21449,0.535320,0.280923,0.254397,0.221071,0.314249,0.214983,0.320337,0.275739,0.259581,0.236031,...,0.281047,0.254273,0.235794,0.299526,0.249747,0.285573,0.237582,0.297738,0.232485,0.302835


In [15]:
#Nifty call to confirm our variables were properly inputted into our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:08:28.749627,163,0.193524,0.227772,0.215562,0.464287,5.221067
1,2,mean_squared_error,0:06:00.526209,115,0.191869,0.229129,0.216004,0.464763,5.225255
2,3,mean_squared_error,0:02:08.908339,41,0.180627,0.234919,0.219009,0.467984,5.210839
3,4,mean_squared_error,0:01:43.736878,34,0.192083,0.228802,0.215947,0.464701,5.228512
4,5,mean_squared_error,0:04:38.374819,92,0.190100,0.228490,0.216477,0.465271,5.228271
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:01:21.764186,29,0.191598,0.229714,0.216077,0.464841,5.215133
246,247,mean_squared_error,0:01:53.418644,40,0.191198,0.229151,0.216184,0.464956,5.230335
247,248,mean_squared_error,0:02:17.624583,49,0.191406,0.228599,0.216128,0.464896,5.205252
248,249,mean_squared_error,0:02:53.244353,62,0.190895,0.230101,0.216265,0.465043,5.227360


In [17]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.102292,0.212598,-0.105198,0.209692,-0.098584,0.216306,-0.109584,0.205306,-0.108231,...,-0.094493,0.220397,-0.109584,0.205306,-0.084682,0.230208,-0.106797,0.208093,-0.099642,0.215248
1,-0.267150,-0.294532,0.027382,-0.324543,0.057393,-0.413557,0.146407,-0.335163,0.068013,-0.272466,...,-0.257924,0.009226,-0.266622,0.000528,-0.277524,0.010374,-0.356965,0.089815,-0.298852,0.031702
2,0.257710,0.322510,0.064800,0.256768,0.000942,0.229179,0.028531,0.317112,0.059402,0.271670,...,0.328483,0.070773,0.282201,0.024491,0.277131,0.019421,0.281943,0.024233,0.259567,0.001857
3,-0.193330,-0.097400,0.095930,-0.101043,0.092287,-0.089837,0.103493,-0.104651,0.088679,-0.104192,...,-0.090732,0.102598,-0.105898,0.087432,-0.080874,0.112456,-0.102980,0.090350,-0.095924,0.097406
4,-0.409560,-0.131309,0.278251,-0.132399,0.277161,-0.128265,0.281295,-0.140663,0.268897,-0.132565,...,-0.113730,0.295830,-0.127115,0.282445,-0.108316,0.301244,-0.132336,0.277224,-0.122928,0.286632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.085033,0.063796,-0.083031,0.061794,-0.065891,0.044654,-0.094501,0.073264,-0.084985,...,-0.074353,0.053116,-0.086325,0.065088,-0.061591,0.040354,-0.084991,0.063754,-0.074971,0.053734
21447,-0.264320,0.100697,0.365017,0.078130,0.342450,0.081205,0.345525,0.095084,0.359404,0.081306,...,0.088925,0.353245,0.067965,0.332285,0.107754,0.372074,0.082551,0.346871,0.092144,0.356464
21448,0.060178,-0.103086,0.163264,-0.106952,0.167130,-0.103130,0.163308,-0.113062,0.173240,-0.108888,...,-0.095098,0.155276,-0.110177,0.170355,-0.085295,0.145473,-0.107426,0.167604,-0.100240,0.160418
21449,0.535320,0.280923,0.254397,0.221071,0.314249,0.214983,0.320337,0.275739,0.259581,0.236031,...,0.281047,0.254273,0.235794,0.299526,0.249747,0.285573,0.237582,0.297738,0.232485,0.302835


In [18]:
model_record.to_csv('data/rnn_results_7_2.csv', index = False)
predict_record.to_csv('data/rnn_predictions_7_2.csv', index = False)